In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image
from sklearn.metrics import multilabel_confusion_matrix, classification_report, roc_curve, auc
import os

from PIL import Image, ImageOps
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
import json

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

In [ ]:
BASE_PATH = "path_to_directory"
dataset = pd.read_csv("path_to_directory")
len(dataset)

In [ ]:
class DenseNet121(nn.Module):
    def __init__(self):
        super(DenseNet121, self).__init__()
        self.densenet121 = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Identity()  # No classifier yet, only features

    def forward(self, x):
        return self.densenet121(x)

class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        self.resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        num_ftrs = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Identity()  # No classifier yet, only features

    def forward(self, x):
        return self.resnet50(x)

class CombinedModel(nn.Module):
    def __init__(self, out_size):
        super(CombinedModel, self).__init__()
        # Instantiate DenseNet121 and ResNet50 for frontal and lateral views
        self.densenet_frontal = DenseNet121()
        self.resnet_frontal = ResNet50()
        
        self.densenet_lateral = DenseNet121()
        self.resnet_lateral = ResNet50()
        
        # The combined feature size
        frontal_feature_size = 1024 + 2048  # Assuming DenseNet121 outputs 1024 and ResNet50 outputs 2048
        lateral_feature_size = 1024 + 2048
        
        combined_feature_size = frontal_feature_size + lateral_feature_size
        
        # Final classifier layer
        self.classifier = nn.Sequential(
            nn.Linear(combined_feature_size, out_size),
            nn.Sigmoid()  # Assuming binary classification for multi-label
        )
        
    def forward(self, x_frontal, x_lateral):
        # Extract features from DenseNet and ResNet for both views
        frontal_features = torch.cat([self.densenet_frontal(x_frontal), self.resnet_frontal(x_frontal)], dim=1)
        lateral_features = torch.cat([self.densenet_lateral(x_lateral), self.resnet_lateral(x_lateral)], dim=1)
        
        # Combine frontal and lateral features
        combined_features = torch.cat([frontal_features, lateral_features], dim=1)
        
        # Final output through classifier
        out = self.classifier(combined_features)
        
        return out

In [ ]:
N_LABELS = 14
# Load the saved model
best_model = CombinedModel(out_size=N_LABELS)
best_model.load_state_dict(torch.load('path_to_best_model.pth'))
best_model = best_model.to(device)
best_model.eval()

In [ ]:
# Load the JSON file with test image paths
with open('test_image_paths.json', 'r') as f:
    test_image_paths = json.load(f)

frontal_images = test_image_paths['frontal_images']
lateral_images = test_image_paths['lateral_images']

# Ensure the two lists of images align by pairing images correctly
assert len(frontal_images) == len(lateral_images), "Frontal and lateral image lists must have the same length."

# Prepare arrays for true and predicted labels
y_true = []
y_pred = []

# Classes
classes = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 
    'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 
    'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'
]

In [ ]:
# Process each pair of frontal and lateral images
for frontal_img_path, lateral_img_path in zip(frontal_images, lateral_images):
    frontal_full_path = f"{BASE_PATH}/{frontal_img_path}"
    lateral_full_path = f"{BASE_PATH}/{lateral_img_path}"

    # Get true labels
    true_labels = dataset[dataset['frontal_image'] == frontal_img_path][classes].values[0]
    y_true.append(true_labels)

    # Load and preprocess images
    frontal_img = Image.open(frontal_full_path).convert('RGB')
    frontal_img_tensor = transform(frontal_img).unsqueeze(0).to(device)
    lateral_img = Image.open(lateral_full_path).convert('RGB')
    lateral_img_tensor = transform(lateral_img).unsqueeze(0).to(device)

    # Predict labels using the model
    with torch.no_grad():
        output = best_model(frontal_img_tensor, lateral_img_tensor)
    y_pred.append(output.cpu().numpy()[0])  # Storing probabilities instead of binary labels

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute and display multilabel confusion matrices
mcm = multilabel_confusion_matrix(y_true > 0.5, y_pred > 0.5)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
for i, class_name in enumerate(classes):
    fpr, tpr, _ = roc_curve(y_true[:, i], y_pred[:, i])
    auc_score = roc_auc_score(y_true[:, i], y_pred[:, i])
    ax.plot(fpr, tpr, label=f'{class_name} (AUC = {auc_score:.2f})')

# Add diagonal line
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
# Add labels and legend
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve for each class")
ax.legend(loc="lower right")
plt.savefig('roc_curves_final.png')
plt.show()